## Sentiment Analysis by Pre-developed Twitter Library -- TextBlob

#### By Joyce Jiang | Code by Joyce

### Citation & Source of my code
Majority of my code is inspired by Siraj Raval's tutorial YouTube video Twitter Sentiment Analysis - Learn Python for Data Science, you can check it out at https://www.youtube.com/watch?v=o_OZdbCzHUA

In [1]:
import nltk
import textblob
import pandas as pd
from pandas import DataFrame
from textblob import TextBlob

In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import string

import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Joyce\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Joyce\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
df= pd.read_excel('Labeled/Covid19UG_NoDuplicates23_05_20_labeled.xlsx',index=False)
#df= pd.read_excel('Labeled/Tweet_Museveni_Sentiment_Labeled_D1.xlsx',index=False)

df

,Unnamed: 0,Row ID,Tweet ID,Username,Tweet,Time,Tweet Type,Retweeted By,Number of Retweets,Hashtags,Mentions,Name,Location,Web,Bio,Number of Tweets,Number of Followers,Number Following,Location Coordinates,SENTIMENT
0,0,1,1263678902041526016,AhmadiyyaUg,His excellency president #Uganda @KagutaMuseve...,5/21/2020 8:51:42 PM,Retweet,AhmadiyyaKings1,4,Uganda Ahmadiyya COVID19UG,KagutaMuseveni HumanityFirstUK UgHumanityfirst...,AhmadiyyaUganda,NaN,NaN,Official Twitter Handle of Uganda Ahmadiyya Mu...,110,155,438,NaN,1.0
1,1,4,1263678714124153088,NansinguzaJ,@Katabasasa @nbstv #NBSFrontline #NBSUpdates\n...,5/21/2020 8:50:58 PM,Tweet,NaN,0,NBSFrontline NBSUpdates COVID19UG Covid_19,Katabasasa nbstv,Nansinguza Jacob,Uganda,https://t.co/8zcFVolmZH,#VisualCommunication #MedicaIillustration & #E...,38917,3300,4103,+01.25+032.5/,NaN
2,2,7,1263677791222074880,SolomonMGrace2,Mr. ERIAS Lukwago should know that SOPs are in...,5/21/2020 8:47:18 PM,Retweet,kugonza6,1,NaN,JudiciaryUG peter_katwesige OfwonoOpondo,Male Solomon Grace,"Kampala, Uganda",https://t.co/CUY7tvkw87,"Public Administrator, Policy Researcher and An...",8851,1246,5009,+00.31628+032.58219/,NaN
3,3,8,1263676429994254080,Cotildakhainza,Energy Minister @DrKitutu hands over the Ugx.2...,5/21/2020 8:41:53 PM,Retweet,h_ssali,18,COVID19UG,DrKitutu Parliament_Ug,Psycho,"Kampala, Uganda",NaN,Student of MAKERERE University Kampala. PURSUI...,1636,1341,1206,+00.31628+032.58219/,NaN
4,4,9,1263674084405252096,matookerepublic,15 new COVID-19 cases have been confirmed by t...,5/21/2020 8:32:34 PM,Retweet,KKabajwisa,1,NaN,NaN,Matooke Republic,"Kampala, Uganda",https://t.co/4au2z8o6u8,Freshly Peeled Info,7186,2516,342,+00.31628+032.58219/,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5056,5056,16824,1260636518072421888,OWC_ug,@OworiSylvia : We have been engaging directly ...,5/13/2020 11:22:22 AM,Retweet,jbogube,2,NaN,OworiSylvia OPMUganda,Wealth Creation Ug,Uganda,https://t.co/QacjAzznSK,Our mission is ''To improve household income t...,2328,3353,715,+01.25+032.5/,NaN
5057,5057,16826,1260636408328356096,OworiSylvia,Major Rubalamira: @OWC_ug has been supplied by...,5/13/2020 11:21:55 AM,Retweet,jbogube,14,NaN,OWC_ug GovUganda OPMUganda MAAIF_Uganda Parlia...,Sylvia Damalie Owori,NaN,NaN,Exec Assistant to the CC of Operation Wealth C...,48,83,107,NaN,NaN
5058,5058,16838,1260630805971374080,i_kagoya,It is going to be very difficult for governmen...,5/13/2020 10:59:40 AM,Retweet,ElliotOrizaarwa,1,NaN,KagutaMuseveni RuhakanaR,Irene Kagoya,Kampala Uganda,NaN,"A lawyer, & Women's Rights Activist, workin...",3826,792,851,+00.31628+032.58219/,NaN
5059,5059,16839,1260630418816225024,JenifaOchwo,For making the #COVID19UG lockdown bearable......,5/13/2020 10:58:07 AM,Retweet,TheLoveDre,1,COVID19UG ThankYouUgandans,NaN,Jenifa Ochwo,Uganda,NaN,Thinker | Faith | Entertainment | Justice | Go...,30887,3221,831,+01.25+032.5/,NaN


In [4]:
df=df[['Tweet','SENTIMENT']]

#### Capturing and defining positive and negative emoji

In [5]:
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)


emoticons = emoticons_happy.union(emoticons_sad)

#### Cleaning text to feed sentiment analytic tool TextBlob

In [7]:
def clean_tweets(tweet):
 
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(tweet)
#after tweepy preprocessing the colon symbol left remain after      #removing mentions
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
#replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
#remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)
#filter using NLTK library append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []
#looping through conditions
    for w in word_tokens:
#check tokens against stop words , emoticons and punctuations
        if w not in stop_words and w not in emoticons and w not in string.punctuation:
            filtered_tweet.append(w)
    return ' '.join(filtered_tweet)
    #print(word_tokens)
    #print(filtered_sentence)return tweet

#### Loop through all tweets in [Tweet] column and apply TextBlob function on each of the tweet

In [8]:
#I delete all URLs in each tweet to clean the data

Analysis=[]

for tweet in df['Tweet']:
    #re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", tweet)
    tweet= re.sub(r"http\S+", "", tweet)
    clean_text = clean_tweets(tweet)
    #print(tweet)
    analysis=TextBlob(clean_text)
    #print(analysis.sentiment)
    
    Analysis.append(analysis.sentiment)

#### Append all analytic results to a list. Append [Polarity] and [Subjectivity] to the dataframe

In [10]:
Polarity=[]
Subjectivity=[]

for x in range(len(df['Tweet'])):
    Polarity.append(Analysis[x].polarity)
    Subjectivity.append(Analysis[x].subjectivity)

In [11]:
#Adding Polarity and Subjectivity columns

se1 = pd.Series(Polarity)
df['Polarity'] = se1.values
se2 = pd.Series(Subjectivity)
df['Subjectivity'] = se2.values

C:\Users\Joyce\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Joyce\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
#Convert [Polarity] to positive (1), negative (-1), and neutral (0) for comparison

prediction = [-1 if v < 0 else 1 if v>0 else 0 for v in df['Polarity']]
df['Prediction']=prediction
df

C:\Users\Joyce\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,Tweet,SENTIMENT,Polarity,Subjectivity,Prediction
0,His excellency president #Uganda @KagutaMuseve...,1.0,0.000000,0.000000,0
1,@Katabasasa @nbstv #NBSFrontline #NBSUpdates\n...,NaN,0.200000,0.633333,1
2,Mr. ERIAS Lukwago should know that SOPs are in...,NaN,-0.300000,0.400000,-1
3,Energy Minister @DrKitutu hands over the Ugx.2...,NaN,0.150000,0.300000,1
4,15 new COVID-19 cases have been confirmed by t...,NaN,0.268182,0.727273,1
...,...,...,...,...,...
5056,@OworiSylvia : We have been engaging directly ...,NaN,0.233333,0.433333,1
5057,Major Rubalamira: @OWC_ug has been supplied by...,NaN,0.062500,0.500000,1
5058,It is going to be very difficult for governmen...,NaN,-0.250000,0.500000,-1
5059,For making the #COVID19UG lockdown bearable......,NaN,0.000000,0.000000,0


#### Comparing TextBlob output with Sentiment Intensity Analyzer output

In [13]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer=SentimentIntensityAnalyzer()

vs=analyzer.polarity_scores("President Museveni: To ensure that people don't borrow masks from each other because they can't afford. The government has decided to provide these masks free of charge to all Ugandans from six years and above. ")
print(vs)

{'neg': 0.0, 'neu': 0.848, 'pos': 0.152, 'compound': 0.7096}


In [14]:
analysis=TextBlob("President Museveni: To ensure that people don't borrow masks from each other because they can't afford. The government has decided to provide these masks free of charge to all Ugandans from six years and above. ")
print(analysis.sentiment)

Sentiment(polarity=0.09166666666666667, subjectivity=0.42500000000000004)


### Computing Accuracy (Overall, Negative, Positive)

In [15]:
#Drop the one with missing values of [SENTIMENT]

df_output=df.dropna(subset=['SENTIMENT'])
df_output=df_output.reset_index(drop=True) #necessary to reset index: for the loop function to work
df_output

,Tweet,SENTIMENT,Polarity,Subjectivity,Prediction
0,His excellency president #Uganda @KagutaMuseve...,1.0,0.000000,0.000000,0
1,"Yes, he is one of the most celebrated Journali...",1.0,0.050000,0.550000,1
2,"@aajtak Social distancing is a myth, anybody c...",-1.0,-0.188889,0.322222,-1
3,#COVID19UG cases have been 'smalolized' from 2...,1.0,0.000000,0.000000,0
4,#Football256 Update\n\n@ProlineFC are unconten...,-1.0,-0.125000,0.375000,-1
...,...,...,...,...,...
66,"#COVID19UG WhatsApp chat both for accurate, tr...",1.0,0.050000,0.566667,1
67,The day more than 317 Members of Parliament we...,-1.0,-0.150000,0.583333,-1
68,"Timely, accurate information plays a critical ...",1.0,-0.033333,0.644444,-1
69,Hello @HonAniteEvelyn it's really interseting ...,-1.0,-0.053333,0.506667,-1


In [16]:
#Check if positive and negative comments are balanced in the [SENTIMENT] column
df.SENTIMENT.value_counts()

 1.0    36
-1.0    35
Name: SENTIMENT, dtype: int64

In [17]:
#Go through the loop to calculate accuracy

n=0

for i in range(len(df_output['SENTIMENT'])):
    if df_output['SENTIMENT'][i]==df_output['Prediction'][i]:
        n+=1
OverallAccuracy = n/len(df_output['SENTIMENT'])*100

In [18]:
#Split dataframe into positive_only and negative_only comments 

positive = df_output[df_output['SENTIMENT']==1]
negative = df_output[df_output['SENTIMENT']==-1]

positive=positive.reset_index(drop=True)
negative=negative.reset_index(drop=True)

#Go through the loop to calculate accuracy for each of the dataframe

ng=0

for i in range(len(negative['SENTIMENT'])):
    if negative['SENTIMENT'][i]==negative['Prediction'][i]:
        ng+=1
NegativeAccuracy = ng/len(negative['SENTIMENT'])*100

p=0

for i in range(len(positive['SENTIMENT'])):
    if positive['SENTIMENT'][i]==positive['Prediction'][i]:
        p+=1
PositiveAccuracy = p/len(positive['SENTIMENT'])*100

In [19]:
print("Overall Accuracy: "+ str(OverallAccuracy))
print("Negative Accuracy: "+ str(NegativeAccuracy))
print("Positive Accuracy: "+ str(PositiveAccuracy))

Overall Accuracy: 52.112676056338024
Negative Accuracy: 37.142857142857146
Positive Accuracy: 66.66666666666666


In [20]:
#import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

df_output

,Tweet,SENTIMENT,Polarity,Subjectivity,Prediction
0,His excellency president #Uganda @KagutaMuseveni recognising efforts of #Ahmadiyya Muslim Community towards #COVID19UG. So grateful to our partners @HumanityFirstUK @UgHumanityfirst @HFI1995 4 support accorded to the ppl of Uganda. @ntvuganda #EidMubarak #EidUlFitr #NBSUpdates https://t.co/CLDFyJrLgx,1.0,0.000000,0.000000,0
1,"Yes, he is one of the most celebrated Journalists of our time. \nYes, he is the reigning @BBC Komla Dumor award winner.\nBut, he is still humble enough to carry video equipment with the rest of us to the frontlines of the #COVID19 field.\nThe lesson is Humility.\n#COVID19UG https://t.co/RlZ8fxG6oh",1.0,0.050000,0.550000,1
2,"@aajtak Social distancing is a myth, anybody can see this now a days, on road, in streets, at local market and at shops, it's very dangerous at this time #Social_Distancing #COVID19UG #aamaadmiparty #delhicm #ndtv #indiatv #hindustantimmes #timesofindia #bbcnews #ZeeNews #dainikbhasker",-1.0,-0.188889,0.322222,-1
3,#COVID19UG cases have been 'smalolized' from 264 to 145 . \n\nWhere is the Luwero Hajji we celebrate?!,1.0,0.000000,0.000000,0
4,#Football256 Update\n\n@ProlineFC are uncontended though they accept to the fact that they've been relegated due to the coronavirus pandemic \n\nhttps://t.co/3luQE42X6e #COVID19UG https://t.co/fZoKnervck,-1.0,-0.125000,0.375000,-1
5,#COVID19UG messed me up... I would be here talking to my dog. https://t.co/1pUwbHpdqz,-1.0,0.000000,0.000000,0
6,I don't think that #COVID19UG should stop elections. I don't think that the millions who vote attend political rallies #KatikkiroOnTheSPot #NTVOnTheSpot,-1.0,0.000000,0.100000,0
7,"#nbsfrontline should change the opposition frontliners. None of them is discussing substance. All of them think President M7 has failed to manage #COVID19UG & are blaming his tea taking skills. If indeed he has failed, why do we have the least cases in the region &amp; no deaths?",-1.0,-0.433333,0.333333,-1
8,For accurate info about #COVID19UG\n@modoanita\n https://t.co/WiD5p6i1GC,1.0,0.400000,0.633333,1
9,@GodberTumushabe @JaneRuth_Aceng Nurses in Soroti also protesting...No clarity on what the Task Force funds are really doing...#COVID19UG,-1.0,0.200000,0.200000,1


In [21]:
#save the entire dataset
df.to_excel('Output/Covid19UG_textblob.xlsx')

In [22]:
#save the one with pre-labeled sentiments
df_output.to_excel('Output/Covid19UG_textblob_52.11%_N37.14%_P66.67%.xlsx')

#### Further reading:
If you want to compare accuracy between multiple sentiment analytic tools (TextBlob and Sentiment Intensity Analyzer), you can check out sentdex's YouTube video Sentiment Analysis in Python with TextBlob and VADER Sentiment (also Dash p.6) at  https://www.youtube.com/watch?time_continue=198&v=qTyj2R-wcks&feature=emb_title. It also introduces other functions contained in TextBlob tools. 